In [21]:
from __future__ import division
import matplotlib.pyplot as plt
import bayesian_changepoint_detection.generate_data as gd
import seaborn
import pandas as pd
import numpy as np
from functools import partial
import matplotlib.cm as cm
import numpy as np
import json
import os
import glob
from os.path import basename,join,dirname
from datetime import datetime
import numpy as np
from scipy.stats import multivariate_normal, norm
from tqdm import tqdm
from bayesian_changepoint_detection.priors import const_prior
from bayesian_changepoint_detection.offline_likelihoods import IndepentFeaturesLikelihood
import bayesian_changepoint_detection.online_likelihoods as online_ll
from bayesian_changepoint_detection.bayesian_models import offline_changepoint_detection 
from bayesian_changepoint_detection.bayesian_models import online_changepoint_detection
from functools import partial
from bayesian_changepoint_detection.hazard_functions import constant_hazard

%matplotlib inline
%load_ext autoreload
%autoreload 2

def drop_time(df: pd.DataFrame):
    if "time" in df:
        df = df.drop(columns=["time"])
    elif "Time" in df:
        df = df.drop(columns=["Time"])
    return df
def drop_constant(df: pd.DataFrame):
    return df.loc[:, (df != df.iloc[0]).any()]


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:


def bocpd_rca(data):
    sparsity = 5 
    epsilon = 1e-7
    
    data = drop_constant(drop_time(data))
    data = data.fillna(method="ffill")
    data = data.fillna(0)
    for c in data.columns:
        data[c] = (data[c] - np.min(data[c])) / (np.max(data[c]) - np.min(data[c]))
    data = data.fillna(method="ffill")
    data = data.fillna(0)
    
    scores = []
    for c in data.columns:
        # take from this: https://github.com/hildensia/bayesian_changepoint_detection/blob/master/Example_Code.ipynb
        R, maxes = online_changepoint_detection(
            data[c],
            partial(constant_hazard, 50),
            online_ll.StudentT(alpha=0.1, beta=.01, kappa=1, mu=0)
        )
        Nw=10
        out = R[Nw,Nw:-1].tolist()
        for i in range(50):
            out[i] = 0
        scores.append((c, max(out)))
    return scores


In [30]:
for data_path in tqdm(glob.glob("../cfm/data/fse-ss/**/1/simple_data.csv")):
    service_metric = basename(dirname(dirname(data_path)))
    case_idx = basename(dirname(data_path))
        
    data = pd.read_csv(data_path)   
    scores = bocpd_rca(data)

    with open(join("debug", f"{service_metric}_{case_idx}.json"), "w") as f:
        f.write(json.dumps(scores))

 45%|████▌     | 9/20 [02:11<02:40, 14.57s/it]


KeyboardInterrupt: 